# Cast creation algorithm

This code first 

In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
# Read in data
roles = pd.read_csv('principal_actors.csv')
data = pd.read_csv('movies_cleaned_dataset_old.csv')

In [126]:
data

,Movie_name,Movie_box_office_revenue,Movie_runtime,Movie_genres,Main_genre,Main_language,Top_genres,Main_country,Main_continent,Plot_summary,...,averageRating,numVotes,Estimated_Budget,IMDb,Oscar_Wins,Nominations,Profit,Inflation_adjusted_profit,nconst,Director_name
0,Ghosts of Mars,14010832,98.0,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","['Thriller', 'Science Fiction', 'Horror', 'Adv...",English Language,Thriller,United States of America,USA,"Set in the second half of the 22nd century, th...",...,4.9,56894.0,28000000,tt0228333,0,0,-13989168,-1.870336e+07,nm0000118,John Carpenter
1,White Of The Eye,0,110.0,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...","['Thriller', 'Erotic thriller', 'Psychological...",English Language,Thriller,United Kingdom,Europe,A series of murders of rich young women throug...,...,6.1,2894.0,0,tt0094320,0,0,0,0.000000e+00,nm0131910,Donald Cammell
2,A Woman in Flames,0,106.0,"{""/m/07s9rl0"": ""Drama""}",Drama,German Language,Other,Germany,Europe,"Eva, an upper class housewife, becomes frustra...",...,5.9,623.0,0,tt0083949,0,0,0,0.000000e+00,nm0885554,Robert van Ackeren
3,The Sorcerer's Apprentice,0,86.0,"{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","['Family Film', 'Fantasy', 'Adventure', 'World...",English Language,Family Film,South Africa,Africa,"Every hundred years, the evil Morgana returns...",...,NaN,NaN,0,NaN,0,0,0,0.000000e+00,NaN,Unknown
4,Little city,0,93.0,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","['Romantic comedy', 'Ensemble Film', 'Comedy-d...",English Language,Drama,United States of America,USA,"Adam, a San Francisco-based artist who works a...",...,5.8,1129.0,0,tt0119548,0,0,0,0.000000e+00,nm0070142,Roberto Benabib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42150,The Ghost Train,0,82.0,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...","['Crime Fiction', 'Thriller', 'Comedy', 'Super...",English Language,Thriller,United Kingdom,Europe,{{plot}} The film opens with a Great Western e...,...,6.2,1582.0,0,tt0033660,0,0,0,0.000000e+00,nm0285962,Walter Forde
42151,Mermaids: The Body Found,0,120.0,"{""/m/07s9rl0"": ""Drama""}",Drama,English Language,Other,United States of America,USA,Two former National Oceanic Atmospheric Admini...,...,NaN,NaN,0,NaN,0,0,0,0.000000e+00,NaN,Unknown
42152,Knuckle,0,96.0,"{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...","['Biographical film', 'Drama', 'Documentary']",English Language,Drama,Ireland,NaN,{{No plot}} This film follows 12 years in the ...,...,6.8,3193.0,0,tt1606259,0,0,0,0.000000e+00,nm3817974,Ian Palmer
42153,The Super Dimension Fortress Macross II: Lover...,0,150.0,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...","['Science Fiction', 'Japanese Movies', 'Advent...",Japanese Language,Science Fiction,Japan,Asia,"The story takes place in the year 2092,The Sup...",...,NaN,NaN,0,NaN,0,0,0,0.000000e+00,NaN,Unknown


In [127]:
 data[data['tconst'] == "tt0468569"]

,Movie_name,Movie_box_office_revenue,Movie_runtime,Movie_genres,Main_genre,Main_language,Top_genres,Main_country,Main_continent,Plot_summary,...,averageRating,numVotes,Estimated_Budget,IMDb,Oscar_Wins,Nominations,Profit,Inflation_adjusted_profit,nconst,Director_name
19714,The Dark Knight,1004558444,153.0,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...","['Crime Fiction', 'Thriller', 'Superhero movie...",Multilingual,Thriller,United States of America,NaN,"In Gotham City, the Joker and his accomplices...",...,9.0,2797575.0,185000000,tt0468569,2,8,819558444,9.011466e+08,nm0634240,Christopher Nolan


# Define genre here

In [128]:
roles

,tconst,ordering,nconst,category,characters,primaryName,birthYear,deathYear
0,tt0000005,1,nm0443482,actor,"[""Blacksmith""]",Charles Kayser,1878,1966
1,tt0000005,2,nm0653042,actor,"[""Assistant""]",John Ott,1850,1931
2,tt0000007,1,nm0179163,actor,\N,James J. Corbett,1866,1933
3,tt0000007,2,nm0183947,actor,\N,Peter Courtney,1867,1896
4,tt0000008,1,nm0653028,actor,"[""Sneezing Man""]",Fred Ott,1860,1936
...,...,...,...,...,...,...,...,...
12181807,tt9916856,4,nm10538647,actress,"[""Kathi""]",Kathrin Knöpfle,\N,\N
12181808,tt9916880,1,nm2784764,actress,"[""Henry""]",Lizzie Waterworth,\N,\N
12181809,tt9916880,2,nm1483166,actor,"[""Rude Ralph"",""Mischievous Mike"",""Jolly Josh""]",Aidan Cook,\N,\N
12181810,tt9916880,3,nm0254176,actress,"[""Moody Margaret""]",Sue Elliott-Nichols,\N,\N


In [50]:
# Get role counts

role_counts = roles.groupby(['nconst', 'category', 'ordering']).size().unstack(fill_value=0)

# Reset index so we can join on 'nconst'
role_counts.reset_index(inplace=True)

# Get actor details
actor_details = roles[['nconst', 'primaryName', 'birthYear', 'deathYear']].drop_duplicates()

# Join the actor details with their role counts
actor_role_df = actor_details.merge(role_counts, on='nconst')

actor_role_df

,nconst,primaryName,birthYear,deathYear,category,1,2,3,4
0,nm0443482,Charles Kayser,1878,1966,actor,1,0,0,0
1,nm0653042,John Ott,1850,1931,actor,0,1,0,0
2,nm0179163,James J. Corbett,1866,1933,actor,2,0,0,0
3,nm0183947,Peter Courtney,1867,1896,actor,0,1,0,0
4,nm0653028,Fred Ott,1860,1936,actor,1,0,0,0
...,...,...,...,...,...,...,...,...,...
1680361,nm13233318,Ganesh Vasant Patil,\N,\N,actor,0,0,1,0
1680362,nm10538641,Yvonne Akins,\N,\N,actor,1,0,0,0
1680363,nm10538650,Anna Fitoussi,\N,\N,actress,0,1,0,0
1680364,nm10538646,Andreas Demmel,\N,\N,actor,0,0,1,0


In [52]:
# Rename columns to represent role counts
actor_role_df.rename(columns={
    'primaryName': 'name',
    1: 'first_roles',
    2: 'second_roles',
    3: 'third_roles',
    4: 'fourth_roles',
}, inplace=True)

In [53]:
display(actor_role_df[actor_role_df['name'] == 'Christian Bale'])

,nconst,name,birthYear,deathYear,category,first_roles,second_roles,third_roles,fourth_roles
60275,nm0000288,Christian Bale,1974,\N,actor,29,9,9,3


In [54]:
# Get total roles
actor_role_df['total_roles'] = actor_role_df['first_roles'] + actor_role_df['second_roles'] + actor_role_df['third_roles'] + actor_role_df['fourth_roles']

In [129]:
# Map genres to map_genres

drama = ['Drama', 'Crime', 'Thriller']
family = ['Family Film', 'Comedy', 'Musical']
action_adventure = ['Action', 'Action/Adventure']
fiction = ['Science Fiction']
horror = ['Horror']
other = ['Short Film', 'Other']

def map_genre(genre):
    if genre in drama:
        return 'Drama'
    elif genre in family:
        return 'Family'
    elif genre in action_adventure:
        return 'Action/Adventure'
    elif genre in fiction:
        return 'Fiction'
    elif genre in horror:
        return 'Horror'
    else:
        return 'Other'


In [130]:
# Map genres
data['Top_genres'] = data['Top_genres'].apply(map_genre)

In [13]:
data.to_csv('movies_cleaned_dataset.csv', index=False)

In [133]:
from scipy.stats import zscore

# Filling missing values in specific columns with the mean of those columns
data['Inflation_adjusted_profit'].fillna(data['Inflation_adjusted_profit'].mean(), inplace=True)
data['averageRating'].fillna(data['averageRating'].mean(), inplace=True)
data['Oscar_Wins'].fillna(data['Oscar_Wins'].mean(), inplace=True)
data['Nominations'].fillna(data['Nominations'].mean(), inplace=True)

# Capping extreme values in the 'Inflation adjusted profit' column
# This reduces the impact of outliers by setting a threshold (90th percentile here)
# Values above this threshold are set to the threshold value itself
cap_threshold = data['Inflation_adjusted_profit'].quantile(0.90) 
data['capped_profit'] = data['Inflation_adjusted_profit'].clip(upper=cap_threshold)

# Calculating the Z-scores (standard scores) for the capped profit, average rating, Oscar wins, and nominations
# This standardizes these features to have a mean of 0 and a standard deviation of 1, aiding in comparison
z_capped_profit = zscore(data['capped_profit'])
z_rating = zscore(data['averageRating'])
z_oscars = zscore(data['Oscar_Wins'])
z_nominations = zscore(data['Nominations'])

# Assigning weights to each of these standardized features
weight_capped_profit = 0.3
weight_rating = 0.35
weight_oscars = 0.175
weight_nominations = 0.175

# Calculating a 'Successful' score based on these weighted features
# This is a composite metric considering profit, rating, Oscars, and nominations
data['Successful'] = (
    weight_capped_profit * z_capped_profit +
    weight_rating * z_rating +
    weight_oscars * z_oscars +
    weight_nominations * z_nominations)

# Normalizing the 'Successful' score to a 0-10 scale
# This makes the score more interpretable and standardized
data['Successful'] = round((data['Successful'] - data['Successful'].min()) / (data['Successful'].max() - data['Successful'].min()) * 10,1)

In [135]:
# Dropping the original profit column   
data = data.drop(columns= ['Movie_runtime', 'Inflation_adjusted_profit', 'Main_language', 'Oscar_Wins', 'Nominations', 'averageRating', 'Main_country', 'Main_continent', 'Plot_summary', 'numVotes', 'Estimated_Budget'])

In [137]:
# Dropping the original profit column
data = data.drop(columns=['Movie_box_office_revenue', 'Main_genre', 'Movie_genres', 'IMDb', 'Profit'])

In [ ]:
display(data[data['Successful'] > 9])

KeyError: 'Successful'

In [62]:

def map_role(tconst):
    # Get the first, second, third, and fourth billed actors
    return roles[(roles['ordering'] == 1) & (roles['tconst'] == tconst)]['nconst'], roles[(roles['ordering'] == 2) & (roles['tconst'] == tconst)]['nconst'], roles[(roles['ordering'] == 3) & (roles['tconst'] == tconst)]['nconst'], roles[(roles['ordering'] == 4) & (roles['tconst'] == tconst)]['nconst']
    

In [63]:
# Get the first, second, third, and fourth billed actors
roles = roles.drop(columns=['category', 'primaryName', 'birthYear', 'deathYear', 'characters'])
roles_pivot = roles.pivot(index='tconst', columns='ordering', values='nconst')

# Rename the columns for clarity
roles_pivot.columns = [f'role_{i}' for i in range(1, 5)]

In [64]:
roles_pivot

,role_1,role_2,role_3,role_4
tconst,,,,
tt0000005,nm0443482,nm0653042,NaN,NaN
tt0000007,nm0179163,nm0183947,NaN,NaN
tt0000008,nm0653028,NaN,NaN,NaN
tt0000009,nm0063086,nm0183823,nm1309758,NaN
tt0000011,nm3692297,NaN,NaN,NaN
...,...,...,...,...
tt9916848,nm5262613,nm3665668,nm8250913,nm8690065
tt9916850,nm5262613,nm3665668,nm8250913,nm8690065
tt9916852,nm5262613,nm3665668,nm8250913,nm8690065


In [65]:
data = data.merge(roles_pivot, on='tconst')
data

,Movie_name,Top_genres,Release_Date,tconst,nconst,Director_name,capped_profit,Successful,role_1,role_2,role_3,role_4
0,Ghosts of Mars,Drama,2001,tt0228333,nm0000118,John Carpenter,-1.870336e+07,3.9,nm0000449,nm0001084,nm0000427,nm0005458
1,White Of The Eye,Drama,1987,tt0094320,nm0131910,Donald Cammell,0.000000e+00,4.5,nm0001418,nm0001550,nm0742165,nm0262543
2,A Woman in Flames,Other,1983,tt0083949,nm0885554,Robert van Ackeren,0.000000e+00,4.4,nm0484727,nm0140649,nm0957193,nm0480718
3,Little city,Drama,1997,tt0119548,nm0070142,Roberto Benabib,0.000000e+00,4.4,nm0000954,nm0000542,nm0068725,nm0255588
4,Henry V,Drama,1989,tt0097499,nm0000110,Kenneth Branagh,2.217323e+06,5.5,nm0000110,nm0001394,nm0791871,nm0488289
...,...,...,...,...,...,...,...,...,...,...,...,...
23891,Guilty as Sin,Drama,1993,tt0107057,nm0001486,Sidney Lumet,1.160929e+07,4.6,nm0000360,nm0000467,nm0002332,nm0912001
23892,Into the Mirror,Other,2003,tt0372937,nm1193005,Sung-ho Kim,0.000000e+00,4.5,nm0949167,nm1047988,nm1092665,nm1029669
23893,Gopi Kishan,Drama,1994,tt0109922,nm0240881,Mukesh Duggal,0.000000e+00,4.4,nm0792911,nm0006433,nm0794364,nm0643350
23894,The Ghost Train,Drama,1941,tt0033660,nm0285962,Walter Forde,0.000000e+00,4.5,nm0039450,nm0613768,nm0365692,nm0615333


In [68]:
actors_df = actor_role_df[['nconst','birthYear']]
display(actors_df)
def merge_birth_year(df, actors_df, role_column):
    return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'birthYear': f'{role_column}_birth_year'})

# Merge for each role
for role in ['role_1', 'role_2', 'role_3', 'role_4']:
    data = merge_birth_year(data, actors_df, role)

,nconst,birthYear
0,nm0443482,1878
1,nm0653042,1850
2,nm0179163,1866
3,nm0183947,1867
4,nm0653028,1860
...,...,...
1680361,nm13233318,\N
1680362,nm10538641,\N
1680363,nm10538650,\N
1680364,nm10538646,\N


/var/folders/qd/996x_2t932b4mbltb5bx_y9r0000gn/T/ipykernel_1307/1909437176.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'nconst_x'} in the result is deprecated and will raise a MergeError in a future version.
  return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'birthYear': f'{role_column}_birth_year'})


In [69]:
data

,Movie_name,Top_genres,Release_Date,tconst,nconst_x,Director_name,capped_profit,Successful,role_1,role_2,role_3,role_4,nconst_y,role_1_birth_year,nconst_x,role_2_birth_year,nconst_y,role_3_birth_year,nconst,role_4_birth_year
0,Ghosts of Mars,Drama,2001,tt0228333,nm0000118,John Carpenter,-1.870336e+07,3.9,nm0000449,nm0001084,nm0000427,nm0005458,nm0000449,1974,nm0001084,1969,nm0000427,1949,nm0005458,1967
1,White Of The Eye,Drama,1987,tt0094320,nm0131910,Donald Cammell,0.000000e+00,4.5,nm0001418,nm0001550,nm0742165,nm0262543,nm0001418,1954,nm0001550,1960,nm0742165,1950,nm0262543,1942
2,A Woman in Flames,Other,1983,tt0083949,nm0885554,Robert van Ackeren,0.000000e+00,4.4,nm0484727,nm0140649,nm0957193,nm0480718,nm0484727,1950,nm0140649,1950,nm0957193,1947,nm0480718,\N
3,Little city,Drama,1997,tt0119548,nm0070142,Roberto Benabib,0.000000e+00,4.4,nm0000954,nm0000542,nm0068725,nm0255588,nm0000954,1962,nm0000542,1964,nm0068725,1932,nm0255588,1922
4,Henry V,Drama,1989,tt0097499,nm0000110,Kenneth Branagh,2.217323e+06,5.5,nm0000110,nm0001394,nm0791871,nm0488289,nm0000110,1960,nm0001394,1938,nm0791871,1956,nm0488289,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,Guilty as Sin,Drama,1993,tt0107057,nm0001486,Sidney Lumet,1.160929e+07,4.6,nm0000360,nm0000467,nm0002332,nm0912001,nm0000360,1959,nm0000467,1949,nm0002332,1952,nm0912001,1920
23892,Into the Mirror,Other,2003,tt0372937,nm1193005,Sung-ho Kim,0.000000e+00,4.5,nm0949167,nm1047988,nm1092665,nm1029669,nm0949167,1976,nm1047988,1972,nm1092665,1980,nm1029669,1955
23893,Gopi Kishan,Drama,1994,tt0109922,nm0240881,Mukesh Duggal,0.000000e+00,4.4,nm0792911,nm0006433,nm0794364,nm0643350,nm0792911,1961,nm0006433,1974,nm0794364,1969,nm0643350,1946
23894,The Ghost Train,Drama,1941,tt0033660,nm0285962,Walter Forde,0.000000e+00,4.5,nm0039450,nm0613768,nm0365692,nm0615333,nm0039450,1900,nm0613768,1907,nm0365692,1892,nm0615333,1908


In [71]:
actors_df = actor_role_df[['nconst','total_roles']]
def merge_total_roles(df, actors_df, role_column):
    return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'total_roles': f'{role_column}_roles_count'})

# Merge for each role
for role in ['role_1', 'role_2', 'role_3', 'role_4']:
    data = merge_total_roles(data, actors_df, role)

/var/folders/qd/996x_2t932b4mbltb5bx_y9r0000gn/T/ipykernel_1307/2637431196.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'nconst_x'} in the result is deprecated and will raise a MergeError in a future version.
  return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'total_roles': f'{role_column}_roles_count'})
/var/folders/qd/996x_2t932b4mbltb5bx_y9r0000gn/T/ipykernel_1307/2637431196.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'nconst_x'} in the result is deprecated and will raise a MergeError in a future version.
  return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'total_roles': f'{role_column}_roles_count'})


In [72]:
actor_role_df['category'] = actor_role_df['category'].apply(lambda x: 'M' if x == 'actor' else 'F')

In [74]:
actors_df = actor_role_df[['nconst','category']]
def merge_total_roles(df, actors_df, role_column):
    return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'category': f'{role_column}_sex'})

# Merge for each role
for role in ['role_1', 'role_2', 'role_3', 'role_4']:
    data = merge_total_roles(data, actors_df, role)

/var/folders/qd/996x_2t932b4mbltb5bx_y9r0000gn/T/ipykernel_1307/471939116.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'nconst_x'} in the result is deprecated and will raise a MergeError in a future version.
  return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'category': f'{role_column}_sex'})
/var/folders/qd/996x_2t932b4mbltb5bx_y9r0000gn/T/ipykernel_1307/471939116.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'nconst_x'} in the result is deprecated and will raise a MergeError in a future version.
  return df.merge(actors_df, left_on = role_column, right_on='nconst', how='left').rename(columns={'category': f'{role_column}_sex'})


In [75]:
data

,Movie_name,Top_genres,Release_Date,tconst,nconst_x,Director_name,capped_profit,Successful,role_1,role_2,...,nconst_x,role_4_roles_count,nconst_y,role_1_sex,nconst_x,role_2_sex,nconst_y,role_3_sex,nconst,role_4_sex
0,Ghosts of Mars,Drama,2001,tt0228333,nm0000118,John Carpenter,-1.870336e+07,3.9,nm0000449,nm0001084,...,nm0005458,53.0,nm0000449,F,nm0001084,M,nm0000427,F,nm0005458,M
1,White Of The Eye,Drama,1987,tt0094320,nm0131910,Donald Cammell,0.000000e+00,4.5,nm0001418,nm0001550,...,nm0262543,25.0,nm0001418,M,nm0001550,F,nm0742165,M,nm0262543,M
2,A Woman in Flames,Other,1983,tt0083949,nm0885554,Robert van Ackeren,0.000000e+00,4.4,nm0484727,nm0140649,...,nm0480718,3.0,nm0484727,F,nm0140649,M,nm0957193,M,nm0480718,F
3,Little city,Drama,1997,tt0119548,nm0070142,Roberto Benabib,0.000000e+00,4.4,nm0000954,nm0000542,...,nm0255588,1.0,nm0000954,M,nm0000542,F,nm0068725,M,nm0255588,M
4,Henry V,Drama,1989,tt0097499,nm0000110,Kenneth Branagh,2.217323e+06,5.5,nm0000110,nm0001394,...,nm0488289,12.0,nm0000110,M,nm0001394,M,nm0791871,M,nm0488289,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,Guilty as Sin,Drama,1993,tt0107057,nm0001486,Sidney Lumet,1.160929e+07,4.6,nm0000360,nm0000467,...,nm0912001,204.0,nm0000360,F,nm0000467,M,nm0002332,M,nm0912001,M
23892,Into the Mirror,Other,2003,tt0372937,nm1193005,Sung-ho Kim,0.000000e+00,4.5,nm0949167,nm1047988,...,nm1029669,17.0,nm0949167,M,nm1047988,M,nm1092665,F,nm1029669,M
23893,Gopi Kishan,Drama,1994,tt0109922,nm0240881,Mukesh Duggal,0.000000e+00,4.4,nm0792911,nm0006433,...,nm0643350,35.0,nm0792911,M,nm0006433,F,nm0794364,F,nm0643350,M
23894,The Ghost Train,Drama,1941,tt0033660,nm0285962,Walter Forde,0.000000e+00,4.5,nm0039450,nm0613768,...,nm0615333,6.0,nm0039450,M,nm0613768,M,nm0365692,F,nm0615333,M


In [78]:
data.rename(columns={data.columns[4]: 'Director_id'},inplace=True)

In [81]:
data.drop(data.columns[[12, 14, 16, 18, 20, 22, 24, 26]],axis = 1, inplace = True)

In [87]:
data

,Movie_name,Top_genres,Release_Date,tconst,Director_name,capped_profit,Successful,role_1,role_2,role_3,...,role_3_birth_year,role_4_birth_year,role_1_roles_count,role_2_roles_count,role_3_roles_count,role_4_roles_count,role_1_sex,role_2_sex,role_3_sex,role_4_sex
0,Ghosts of Mars,Drama,2001,tt0228333,John Carpenter,-1.870336e+07,3.9,nm0000449,nm0001084,nm0000427,...,1949,1967,133.0,43.0,52.0,53.0,F,M,F,M
1,White Of The Eye,Drama,1987,tt0094320,Donald Cammell,0.000000e+00,4.5,nm0001418,nm0001550,nm0742165,...,1950,1942,68.0,40.0,115.0,25.0,M,F,M,M
2,A Woman in Flames,Other,1983,tt0083949,Robert van Ackeren,0.000000e+00,4.4,nm0484727,nm0140649,nm0957193,...,1947,\N,76.0,232.0,96.0,3.0,F,M,M,F
3,Little city,Drama,1997,tt0119548,Roberto Benabib,0.000000e+00,4.4,nm0000954,nm0000542,nm0068725,...,1932,1922,21.0,65.0,5.0,1.0,M,F,M,M
4,Henry V,Drama,1989,tt0097499,Kenneth Branagh,2.217323e+06,5.5,nm0000110,nm0001394,nm0791871,...,1956,\N,148.0,187.0,30.0,12.0,M,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,Guilty as Sin,Drama,1993,tt0107057,Sidney Lumet,1.160929e+07,4.6,nm0000360,nm0000467,nm0002332,...,1952,1920,51.0,299.0,78.0,204.0,F,M,M,M
23892,Into the Mirror,Other,2003,tt0372937,Sung-ho Kim,0.000000e+00,4.5,nm0949167,nm1047988,nm1092665,...,1980,1955,60.0,33.0,11.0,17.0,M,M,F,M
23893,Gopi Kishan,Drama,1994,tt0109922,Mukesh Duggal,0.000000e+00,4.4,nm0792911,nm0006433,nm0794364,...,1969,1946,109.0,63.0,793.0,35.0,M,F,F,M
23894,The Ghost Train,Drama,1941,tt0033660,Walter Forde,0.000000e+00,4.5,nm0039450,nm0613768,nm0365692,...,1892,1908,24.0,11.0,41.0,6.0,M,M,F,M


In [88]:
data.to_csv('cast_processed_corrected_genres.csv', index=False)

In [35]:
display(data[(data['Top_genres'] != 'Drama') & (data['Top_genres'] != 'Other')])

,Movie_name,Top_genres,Release_Date,tconst,capped_profit,Successful,role_1,role_2,role_3,role_4,...,role_3_birth_year,role_4_birth_year,role_1_roles_count,role_2_roles_count,role_3_roles_count,role_4_roles_count,role_1_sex,role_2_sex,role_3_sex,role_4_sex
15,The Gods Must Be Crazy,Action/Adventure,1980,tt0080801,1.160929e+07,4.9,nm0618452,nm0923366,nm0697865,nm0895116,...,\N,1930,5.0,38.0,73.0,8.0,M,M,F,M
17,Kinjite: Forbidden Subjects,Action/Adventure,1989,tt0097670,6.525069e+06,4.5,nm0000314,nm0273592,nm0520164,nm0657070,...,1929,\N,126.0,19.0,16.0,8.0,M,M,M,M
21,The Human Tornado,Action/Adventure,1976,tt0074653,0.000000e+00,4.5,nm0601834,nm0715539,nm0528345,nm0413595,...,1937,1951,8.0,2.0,4.0,3.0,M,F,M,M
25,Nariman,Action/Adventure,2001,tt0323365,0.000000e+00,4.3,nm0329730,nm1388202,nm0415538,nm0415549,...,1955,1951,130.0,116.0,95.0,129.0,M,M,M,M
26,Drums in the Deep South,Action/Adventure,1951,tt0043482,0.000000e+00,4.4,nm0185883,nm0668510,nm0534972,nm0533692,...,1922,1902,50.0,10.0,52.0,81.0,M,F,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23840,Pilots,Action/Adventure,2000,tt0282843,0.000000e+00,3.9,nm0329730,nm0820208,nm0695595,nm0415549,...,1978,1951,130.0,82.0,2653.0,129.0,M,M,F,M
23861,Navajo Joe,Action/Adventure,1966,tt0061587,0.000000e+00,4.5,nm0000608,nm0759757,nm0532609,nm0721073,...,1944,1917,300.0,35.0,23.0,152.0,M,M,F,M
23868,Twister,Action/Adventure,1996,tt0117998,1.160929e+07,4.8,nm0000166,nm0000200,nm0000144,nm0000415,...,1962,1965,237.0,129.0,86.0,98.0,F,M,M,F
23870,The Four Feathers,Action/Adventure,1939,tt0031334,0.000000e+00,4.7,nm0166234,nm0724732,nm0807580,nm0243454,...,1863,1918,15.0,88.0,37.0,10.0,M,M,M,F


In [104]:
actor_role_df['birthYear'] = actor_role_df['birthYear'].fillna("0")
actor_role_df['birthYear'] = actor_role_df['birthYear'].replace("\\N", "0")
actor_role_df['age'] = 2023 - actor_role_df['birthYear'].astype(int)

In [105]:
actor_role_df

,nconst,name,birthYear,deathYear,category,first_roles,second_roles,third_roles,fourth_roles,total_roles,age
0,nm0443482,Charles Kayser,1878,1966,M,1,0,0,0,1,145
1,nm0653042,John Ott,1850,1931,M,0,1,0,0,1,173
2,nm0179163,James J. Corbett,1866,1933,M,2,0,0,0,2,157
3,nm0183947,Peter Courtney,1867,1896,M,0,1,0,0,1,156
4,nm0653028,Fred Ott,1860,1936,M,1,0,0,0,1,163
...,...,...,...,...,...,...,...,...,...,...,...
1680361,nm13233318,Ganesh Vasant Patil,0,\N,M,0,0,1,0,1,2023
1680362,nm10538641,Yvonne Akins,0,\N,M,1,0,0,0,1,2023
1680363,nm10538650,Anna Fitoussi,0,\N,F,0,1,0,0,1,2023
1680364,nm10538646,Andreas Demmel,0,\N,M,0,0,1,0,1,2023


In [111]:
actors_df_alive = actor_role_df[(actor_role_df['deathYear'] == '\\N') & (actor_role_df['age'] < 105)]


In [112]:
actors_df_alive

,nconst,name,birthYear,deathYear,category,first_roles,second_roles,third_roles,fourth_roles,total_roles,age
1845,nm0209802,Luis de Llano,1945,\N,M,0,1,0,0,1,78
5228,nm0317854,Pilar Gil,1977,\N,F,1,3,0,2,6,46
5252,nm0869732,Cecilia Toussaint,1958,\N,F,4,2,4,0,10,65
5254,nm0650495,Regina Orozco,1964,\N,F,1,4,6,5,16,59
7597,nm1009424,Judi Gassel,1927,\N,F,2,1,0,0,3,96
...,...,...,...,...,...,...,...,...,...,...,...
1680267,nm1320801,Nate Miller,1976,\N,M,0,1,0,0,1,47
1680268,nm1665443,Mikhail Solodko,1967,\N,M,0,0,0,1,1,56
1680279,nm1832009,Ahmet Faik Akinci,1956,\N,M,1,0,0,0,1,67
1680343,nm0508273,Rosa León,1951,\N,F,0,0,2,0,2,72


In [113]:
actors_df_alive.to_csv('actors_alive.csv', index=False)

In [123]:
data

,Movie_name,Top_genres,Release_Date,tconst,Director_name,capped_profit,Successful,role_1,role_2,role_3,...,role_3_birth_year,role_4_birth_year,role_1_roles_count,role_2_roles_count,role_3_roles_count,role_4_roles_count,role_1_sex,role_2_sex,role_3_sex,role_4_sex
0,Ghosts of Mars,Drama,2001,tt0228333,John Carpenter,-1.870336e+07,3.9,nm0000449,nm0001084,nm0000427,...,1949,1967,133.0,43.0,52.0,53.0,F,M,F,M
1,White Of The Eye,Drama,1987,tt0094320,Donald Cammell,0.000000e+00,4.5,nm0001418,nm0001550,nm0742165,...,1950,1942,68.0,40.0,115.0,25.0,M,F,M,M
2,A Woman in Flames,Other,1983,tt0083949,Robert van Ackeren,0.000000e+00,4.4,nm0484727,nm0140649,nm0957193,...,1947,\N,76.0,232.0,96.0,3.0,F,M,M,F
3,Little city,Drama,1997,tt0119548,Roberto Benabib,0.000000e+00,4.4,nm0000954,nm0000542,nm0068725,...,1932,1922,21.0,65.0,5.0,1.0,M,F,M,M
4,Henry V,Drama,1989,tt0097499,Kenneth Branagh,2.217323e+06,5.5,nm0000110,nm0001394,nm0791871,...,1956,\N,148.0,187.0,30.0,12.0,M,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,Guilty as Sin,Drama,1993,tt0107057,Sidney Lumet,1.160929e+07,4.6,nm0000360,nm0000467,nm0002332,...,1952,1920,51.0,299.0,78.0,204.0,F,M,M,M
23892,Into the Mirror,Other,2003,tt0372937,Sung-ho Kim,0.000000e+00,4.5,nm0949167,nm1047988,nm1092665,...,1980,1955,60.0,33.0,11.0,17.0,M,M,F,M
23893,Gopi Kishan,Drama,1994,tt0109922,Mukesh Duggal,0.000000e+00,4.4,nm0792911,nm0006433,nm0794364,...,1969,1946,109.0,63.0,793.0,35.0,M,F,F,M
23894,The Ghost Train,Drama,1941,tt0033660,Walter Forde,0.000000e+00,4.5,nm0039450,nm0613768,nm0365692,...,1892,1908,24.0,11.0,41.0,6.0,M,M,F,M


In [139]:
data.columns

Index(['Movie_name', 'Top_genres', 'Release_Date', 'tconst', 'nconst',
       'Director_name', 'capped_profit', 'Successful'],
      dtype='object')

In [121]:
directors = pd.read_csv('directors.csv')
directors
# movie name, release date, top genres, nconst, director name, death year, nb movies, director age, successful, 

,nconst,director_name,num_movies,movies,birth_year,death_year
0,nm0000005,Ingmar Bergman,85,"['tt0038468', 'tt0038675', 'tt0039834', 'tt004...",1918,2007
1,nm0000008,Marlon Brando,1,['tt0055257'],1924,2004
2,nm0000009,Richard Burton,1,['tt0062898'],1925,1984
3,nm0000010,James Cagney,1,['tt0050964'],1899,1986
4,nm0000018,Kirk Douglas,1,['tt0073559'],1916,2020
...,...,...,...,...,...,...
686037,nm9993679,Art Jones,1,['tt8744074'],\N,\N
686038,nm9993694,Chinmay Mishra,2,"['tt18361688', 'tt18687502']",\N,\N
686039,nm9993696,Ibrahim-Aloduley,1,['tt8744160'],\N,\N
686040,nm9993708,Eli Bevins,9,"['tt11702702', 'tt11753904', 'tt11772812', 'tt...",\N,\N


In [140]:
directors.drop(columns="movies", inplace=True)
data2 = data.drop(columns='capped_profit')

In [142]:
directors

,nconst,director_name,num_movies,birth_year,death_year
0,nm0000005,Ingmar Bergman,85,1918,2007
1,nm0000008,Marlon Brando,1,1924,2004
2,nm0000009,Richard Burton,1,1925,1984
3,nm0000010,James Cagney,1,1899,1986
4,nm0000018,Kirk Douglas,1,1916,2020
...,...,...,...,...,...
686037,nm9993679,Art Jones,1,\N,\N
686038,nm9993694,Chinmay Mishra,2,\N,\N
686039,nm9993696,Ibrahim-Aloduley,1,\N,\N
686040,nm9993708,Eli Bevins,9,\N,\N


In [143]:
data2 = data2.merge(directors, on='nconst', how='left')

In [149]:
data2.dropna(subset = 'birth_year',inplace=True)
data2.replace("\\N", "0", inplace=True)
data2['Director_age'] = data2['Release_Date'] - data2['birth_year'].astype(int)

In [152]:
data2 = data2[data2['Director_age'] < 105]
data2

,Movie_name,Top_genres,Release_Date,tconst,nconst,Director_name,Successful,director_name,num_movies,birth_year,death_year,Director_age
0,Ghosts of Mars,Drama,2001,tt0228333,nm0000118,John Carpenter,3.9,John Carpenter,32.0,1948,0,53
1,White Of The Eye,Drama,1987,tt0094320,nm0131910,Donald Cammell,4.5,Donald Cammell,5.0,1934,1996,53
2,A Woman in Flames,Other,1983,tt0083949,nm0885554,Robert van Ackeren,4.4,Robert van Ackeren,14.0,1946,0,37
5,Henry V,Drama,1989,tt0097499,nm0000110,Kenneth Branagh,5.5,Kenneth Branagh,23.0,1960,0,29
8,Mary Poppins,Family,1964,tt0058331,nm0829038,Robert Stevenson,8.1,Robert Stevenson,134.0,1905,1986,59
...,...,...,...,...,...,...,...,...,...,...,...,...
42141,Mirage,Drama,1972,tt0068955,nm0324162,Armando Robles Godoy,4.6,Armando Robles Godoy,9.0,1923,2010,49
42143,The Flying Serpent,Drama,1946,tt0038531,nm0627864,Sam Newfield,4.2,Sam Newfield,324.0,1899,1964,47
42145,State and Main,Family,2000,tt0120202,nm0000519,David Mamet,4.8,David Mamet,25.0,1947,0,53
42146,Guilty as Sin,Drama,1993,tt0107057,nm0001486,Sidney Lumet,4.6,Sidney Lumet,154.0,1924,2011,69


In [153]:
data2.to_csv('movies_directors_wit h_genre.csv', index=False)

In [165]:
actor_role_df[actor_role_df['nconst'] == 'nm5397459']

,nconst,name,birthYear,deathYear,category,first_roles,second_roles,third_roles,fourth_roles,total_roles,age
361349,nm5397459,Daisy Ridley,1992,\N,F,24,9,7,0,40,31


In [164]:
import re

file_name = 'top_1000.txt' 

try:
    with open(file_name, 'r') as file:
        data = file.read()
    message = "File read successfully."
except FileNotFoundError:
    data = ""
    message = f"File not found: {file_name}"


# Splitting the data into lines
lines = data.strip().split('\n')

# Extracting IMDb nconst using regular expression
nconsts = []
for line in lines:
    match = re.search(r'nm\d+', line)
    if match:
        nconsts.append(match.group())

# Creating a pandas DataFrame
df = pd.DataFrame(nconsts, columns=['IMDb nconst'])
df


,IMDb nconst
0,nm2394794
1,nm0249291
2,nm0636426
3,nm5397459
4,nm4867258
...,...
990,nm0004757
991,nm0403652
992,nm0005346
993,nm0947338


In [167]:
top1000 = df.merge(actor_role_df, left_on='IMDb nconst', right_on='nconst', how='left')
top1000.sort_values(by='age', ascending=False, inplace=True)

In [173]:
top1000.dropna(subset=['age'], inplace=True)
top1000 = top1000[top1000['age'] < 95]
top1000[top1000['deathYear'] == '\\N']

,IMDb nconst,nconst,name,birthYear,deathYear,category,first_roles,second_roles,third_roles,fourth_roles,total_roles,age
225,nm0000142,nm0000142,Clint Eastwood,1930,\N,M,59.0,203.0,2.0,2.0,266.0,93.0
730,nm0000323,nm0000323,Michael Caine,1933,\N,M,70.0,24.0,15.0,9.0,118.0,90.0
970,nm0001132,nm0001132,Judi Dench,1934,\N,F,154.0,42.0,14.0,10.0,220.0,89.0
466,nm0001749,nm0001749,Maggie Smith,1934,\N,F,18.0,14.0,14.0,4.0,50.0,89.0
798,nm0002663,nm0002663,Diane Ladd,1935,\N,F,11.0,19.0,43.0,20.0,93.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...
753,nm5897057,nm5897057,Jaeden Martell,2003,\N,M,3.0,8.0,10.0,3.0,24.0,20.0
36,nm5611121,nm5611121,Millie Bobby Brown,2004,\N,F,10.0,7.0,15.0,25.0,57.0,19.0
860,nm6748436,nm6748436,Dafne Keen,2005,\N,F,17.0,4.0,10.0,1.0,32.0,18.0
718,nm5085683,nm5085683,Mckenna Grace,2006,\N,F,32.0,6.0,6.0,6.0,50.0,17.0


In [175]:
top1000 = top1000[top1000['deathYear'] == '\\N']

In [178]:
top1000[top1000['category'] == 'M']

,IMDb nconst,nconst,name,birthYear,deathYear,category,first_roles,second_roles,third_roles,fourth_roles,total_roles,age
225,nm0000142,nm0000142,Clint Eastwood,1930,\N,M,59.0,203.0,2.0,2.0,266.0,93.0
730,nm0000323,nm0000323,Michael Caine,1933,\N,M,70.0,24.0,15.0,9.0,118.0,90.0
747,nm0000095,nm0000095,Woody Allen,1935,\N,M,39.0,4.0,1.0,1.0,45.0,88.0
792,nm0000164,nm0000164,Anthony Hopkins,1937,\N,M,63.0,36.0,13.0,7.0,119.0,86.0
538,nm0000151,nm0000151,Morgan Freeman,1937,\N,M,58.0,35.0,291.0,144.0,528.0,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...
366,nm4055138,nm4055138,Charlie Plummer,1999,\N,M,13.0,4.0,2.0,16.0,35.0,24.0
417,nm7140802,nm7140802,Gaten Matarazzo,2002,\N,M,1.0,1.0,0.0,0.0,2.0,21.0
176,nm6016511,nm6016511,Finn Wolfhard,2002,\N,M,4.0,35.0,27.0,13.0,79.0,21.0
753,nm5897057,nm5897057,Jaeden Martell,2003,\N,M,3.0,8.0,10.0,3.0,24.0,20.0


In [179]:
# saving the dataframe
top1000.to_csv('top1000_actors.csv', index=False)